# 如何對資料框進行基礎操作

> 寫一些 pandas 語法操作 Johns Hopkins COVID-19 每日報告
>
> 標籤：程式設計，獲取載入，整併轉換

郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
# 載入專案需要使用的套件
import datetime
import pandas as pd

## TL; DR

在這個專案中，我們打算寫一些 pandas 語法操作約翰霍普金斯大學 COVID-19 Data Repository 中最新的每日報告，讀者將學會如何定義函式 `get_latest_daily_report()` 將約翰霍普金斯大學 COVID-19 Data Repository 中最新的每日報告載入成為資料框、如何衍生新的欄位（衍生治療中案例數）、選擇特定欄位（選擇一個或多個）、篩選指定觀測值（台灣在哪裡）、分組摘要（以國家為單位聚合確診數）以及排序（將摘要結果由大到小遞減排序）。

## 資料來源

資料來源是 [COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19) 中的每日報告資料夾 `/csse_covid_19_data/csse_covid_19_daily_reports`，該資料夾從 2020-01-22 開始每天都有一個單獨的 CSV 檔案記錄該日的全球現況。

<img src="img/daily_report_folder.png">

## 載入最新的每日報告

CSV 檔案的命名是以 `%m-%d-%Y` Unix 時間格式、俗稱的 `mm-dd-yyyy` 格式作為檔案名稱，如果希望載入最新的每日報告，我們可以用電腦的當天日期作為檔名，但是由於資料源更新時間、時區的差異，使用當天日期很有可能沒有對應的檔案，因此我們可以寫一段程式，他的處理邏輯是：

1. 先以電腦的當天日期作為檔名是否可以載入成功
2. 如果成功這段程式的任務就完成了
3. 如果載入失敗產生錯誤訊息，就將當天日期減去 1，直到載入成功

這段程式需要 Python 的標準套件 `datetime`、第三方套件 `pandas`、`try...except...` 語法以及 `while` 語法。其中 `datetime` 可以協助我們獲得電腦的當天日期、進行日期的運算以及調整日期的文字格式。

In [2]:
latest_date = datetime.date.today()
latest_date_fmt = latest_date.strftime('%m-%d-%Y')
print(latest_date_fmt)

09-06-2020


第三方套件 `pandas` 可以協助我們將 CSV 檔案讀入成為方便摘要分析的 `DataFrame`，每日報告由於更新時間、時區差異的緣故，大概都會是昨天或者前天，因此如果貿然將當天日期作為檔名通常會獲得錯誤訊息。這時我們就可以利用 `try...except...` 將錯誤捕捉起來。

In [3]:
csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(latest_date_fmt)
try:
    daily_report = pd.read_csv(csv_url)
except:
    print("尚未有 {} 的每日報告。".format(latest_date_fmt))

尚未有 09-06-2020 的每日報告。


最後加入 `while` 語法，目的是只要錯誤被捕捉起來，就將當天日期減 1 成為再前一天日期，再嘗試一次載入，假若再有錯誤被捕捉，就持續減去 1 天，直到成功為止。

In [4]:
latest_date = datetime.date.today()
day_delta = datetime.timedelta(days=1)
fmt = '%m-%d-%Y'
while True:
    try:
        latest_date_fmt = latest_date.strftime(fmt)
        csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(latest_date_fmt)
        daily_report = pd.read_csv(csv_url)
        print("載入了 {} 的每日報告。".format(latest_date_fmt))
        break
    except:
        latest_date_fmt = latest_date.strftime(fmt)
        print("尚未有 {} 的每日報告。".format(latest_date_fmt))
        latest_date -= day_delta

尚未有 09-06-2020 的每日報告。
載入了 09-05-2020 的每日報告。


## 將載入最新的每日報告包裝成函式

將前面的程式包裝成函式，可以回傳最新每日報告以及檔案日期。

In [5]:
def get_latest_daily_report():
    """
    This function returns the latest global daily report from https://github.com/CSSEGISandData/COVID-19 and its file date.
    """
    latest_date = datetime.date.today()
    day_delta = datetime.timedelta(days=1)
    fmt = '%m-%d-%Y'
    while True:
        try:
            latest_date_fmt = latest_date.strftime(fmt)
            csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(latest_date_fmt)
            daily_report = pd.read_csv(csv_url)
            print("載入了 {} 的每日報告。".format(latest_date_fmt))
            break
        except:
            latest_date_fmt = latest_date.strftime(fmt)
            print("尚未有 {} 的每日報告。".format(latest_date_fmt))
            latest_date -= day_delta
    return latest_date, daily_report

In [6]:
latest_date, daily_report = get_latest_daily_report()

尚未有 09-06-2020 的每日報告。
載入了 09-05-2020 的每日報告。


In [7]:
daily_report.shape # 每日報告的外觀

(3954, 14)

In [8]:
daily_report.head() # 每日報告的前五列

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-09-06 04:28:33,33.93911,67.709953,38324,1409,30082,6833.0,Afghanistan,98.447555,3.676547
1,NaN,NaN,NaN,Albania,2020-09-06 04:28:33,41.15330,20.168300,10102,312,5976,3814.0,Albania,351.032038,3.088497
2,NaN,NaN,NaN,Algeria,2020-09-06 04:28:33,28.03390,1.659600,46071,1549,32481,12041.0,Algeria,105.062495,3.362202
3,NaN,NaN,NaN,Andorra,2020-09-06 04:28:33,42.50630,1.521800,1215,53,928,234.0,Andorra,1572.510192,4.362140
4,NaN,NaN,NaN,Angola,2020-09-06 04:28:33,-11.20270,17.873900,2935,117,1192,1626.0,Angola,8.930129,3.986371


## 衍生新的欄位

治療中案例數可以用既有的欄位計算而得，常見的定義為：確診數減去死亡數再減痊癒數。

\begin{equation}
\text{Active} = \text{Confirmed} - \text{Deaths} - \text{Recovered}
\end{equation}

In [9]:
active = daily_report["Confirmed"] - daily_report["Deaths"] - daily_report["Recovered"]
active

0        6833
1        3814
2       12041
3         234
4        1626
        ...  
3949     8737
3950        1
3951      214
3952      749
3953     1286
Length: 3954, dtype: int64

與原本資料框中的欄位 Active 比對可以檢查每日報告中有問題的觀測值紀錄。

In [10]:
print((active != daily_report["Active"]).sum()) # 衍生欄位與原有欄位 Active 不同的觀測值數有幾筆
daily_report[active != daily_report["Active"]]  # 問題的觀測值紀錄

4


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
67,NaN,NaN,Diamond Princess,Canada,2020-09-06 04:28:33,NaN,NaN,0,1,0,NaN,"Diamond Princess, Canada",NaN,NaN
96,NaN,NaN,Unknown,Chile,2020-09-06 04:28:33,NaN,NaN,54,1,54,NaN,"Unknown, Chile",NaN,1.851852
740,90004.0,Unassigned,Arizona,US,2020-09-06 04:28:33,NaN,NaN,1,1,0,NaN,"Unassigned, Arizona, US",NaN,100.000000
1517,19159.0,Ringgold,Iowa,US,2020-09-06 04:28:33,40.735189,-94.243685,34,1,0,32.0,"Ringgold, Iowa, US",694.728239,2.941176


在資料框的第 0 欄插入衍生欄位。

In [11]:
daily_report.insert(0, 'Derived_Active', active)
daily_report.head()

,Derived_Active,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,6833,NaN,NaN,NaN,Afghanistan,2020-09-06 04:28:33,33.93911,67.709953,38324,1409,30082,6833.0,Afghanistan,98.447555,3.676547
1,3814,NaN,NaN,NaN,Albania,2020-09-06 04:28:33,41.15330,20.168300,10102,312,5976,3814.0,Albania,351.032038,3.088497
2,12041,NaN,NaN,NaN,Algeria,2020-09-06 04:28:33,28.03390,1.659600,46071,1549,32481,12041.0,Algeria,105.062495,3.362202
3,234,NaN,NaN,NaN,Andorra,2020-09-06 04:28:33,42.50630,1.521800,1215,53,928,234.0,Andorra,1572.510192,4.362140
4,1626,NaN,NaN,NaN,Angola,2020-09-06 04:28:33,-11.20270,17.873900,2935,117,1192,1626.0,Angola,8.930129,3.986371


## 選擇特定欄位

在中括號裡頭輸入欄位名稱可以將資料以 `Series` 外型從資料框中取出。

In [12]:
daily_report["Country_Region"]

0              Afghanistan
1                  Albania
2                  Algeria
3                  Andorra
4                   Angola
               ...        
3949    West Bank and Gaza
3950        Western Sahara
3951                 Yemen
3952                Zambia
3953              Zimbabwe
Name: Country_Region, Length: 3954, dtype: object

若想要選擇多個欄位，將多個欄位名稱以 `list` 傳入中括號。

In [13]:
multiple_columns = ["Country_Region", "Confirmed", "Deaths", "Recovered"]
daily_report[multiple_columns]

,Country_Region,Confirmed,Deaths,Recovered
0,Afghanistan,38324,1409,30082
1,Albania,10102,312,5976
2,Algeria,46071,1549,32481
3,Andorra,1215,53,928
4,Angola,2935,117,1192
...,...,...,...,...
3949,West Bank and Gaza,25575,177,16661
3950,Western Sahara,10,1,8
3951,Yemen,1983,572,1197
3952,Zambia,12709,292,11668


## 篩選指定觀測值

在中括號裡頭輸入判斷條件所獲得的布林值 `Series` 可以獲得指定的觀測值。

In [14]:
is_tw = daily_report['Country_Region'] == 'Taiwan*' # 台灣在哪裡
is_tw

0       False
1       False
2       False
3       False
4       False
        ...  
3949    False
3950    False
3951    False
3952    False
3953    False
Name: Country_Region, Length: 3954, dtype: bool

In [15]:
daily_report[is_tw] # is_tw 是一個布林值 Series

,Derived_Active,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
622,12,NaN,NaN,NaN,Taiwan*,2020-09-06 04:28:33,23.7,121.0,492,7,473,12.0,Taiwan*,2.065771,1.422764


## 分組摘要

以國家為單位，所以在 `groupby()` 方法中傳入 Country_Region 欄位，獲得一個 `DataFrameGroupBy` 類別。

In [16]:
daily_report.groupby("Country_Region")

指定 `DataFrameGroupBy` 類別的欄位與聚合函式，獲得分組摘要的結果，是一個 `Series`。

In [17]:
daily_report.groupby("Country_Region")['Confirmed'].sum()

Country_Region
Afghanistan           38324
Albania               10102
Algeria               46071
Andorra                1215
Angola                 2935
                      ...  
West Bank and Gaza    25575
Western Sahara           10
Yemen                  1983
Zambia                12709
Zimbabwe               6837
Name: Confirmed, Length: 188, dtype: int64

## 排序

呼叫 `Series` 的 `sort_values(ascending=False)` 方法將摘要結果由大到小遞減排序。

In [18]:
confirmed_by_country = daily_report.groupby("Country_Region")['Confirmed'].sum()
confirmed_by_country.sort_values(ascending=False)[:10] # 顯示確診人數前 10 高的國家

Country_Region
US              6244970
Brazil          4123000
India           4113811
Russia          1017131
Peru             676848
Colombia         650055
South Africa     636884
Mexico           629409
Spain            498989
Argentina        471806
Name: Confirmed, dtype: int64